# [3] GPT

- 본 섹션은 OpenAI API를 활용하여 GPT 모델을 통해 예측을 수행합니다.

## 3.1. OpenAI developer platform 시작하기

### 3.1.1. 가입하기

https://platform.openai.com/

위 링크로 접속 후 우측 상단 [Sign up] 으로 가입 진행

- 이메일로 가입 또는 계정 연동 로그인 (Google, MS, Apple)
- Full Name, Birthday 입력 (Organization name 생략)


### 3.1.2. 메뉴 구성
- Playground : GUI 기반으로 API를 테스트 해 볼 수 있음
- Dashboard : 자원, 모델 등 관리
- Docs : 시작 가이드 문서
- API reference : 상세 API 문서

### 3.1.3. 조직, 프로젝트, 멤버

- 좌측 상단에 `조직명 / 프로젝트명` 으로 표시 및 선택 가능

- 조직 (Organization)
  - 계정별로 하나씩 부여되는 루트 디렉토리
  - 처음 가입시 조직 이름이 `Personal` 로 설정 됨

- 프로젝트 (project)
  - 조직 하위의 소속의 프로젝트
  - 각 조직에는 `Default project`가 기본으로 생성 됨
  - 프로젝트 생성 방법 : 우측 상단 프로젝트명 클릭 > `+Create project`
  - ⚠️ `Default project` 에서는 가급적 작업하지 말 것! 왜냐하면 조직에 속한 모든 멤버가 `Default project`에 대한 접근 권한이 있어서 보안상 좋지 않음.

- 멤버 (Member) 관리
  1. 조직에 추가 : 조직 선택 > 세팅 > ORGANIZATION > Members > `+Invite`
    - 조직 초대 수락 방법 : 초대 이메일 확인 후 수락
  2. 프로젝트에 추가 : 프로젝트 선택 > 세팅 > PROJECT > Members > `+Add Member`





### 3.1.4. 비용 (Billing)

세팅 > ORGANIZATION > Billing

- Overview : 남은 크래딧 관리
  - Auto recharge : 남은 크래딧이 부족할 경우 자동 충전
- Payment methods : 카드 결제 정보 관리
- Billing history : 결제 내역 관리
- Preferences : 사업자 정보 관리

### 3.1.5. Limits 과 Usage

- Limit 설정
  - 조직 수준 : 세팅 > ORGANIZATION > Limits
  - 프로젝트 수준 : 세팅 > PROJECT > Limits

- Rate limits
  - TPM : 분당 최대 토큰 수 (Tokens Per Minute)
  - TPD : 하루 최대 토큰 수 (Tokens Per Day)
  - RPM : 분당 최대 요청 수 (Requests Per Minute)
  - RPD : 하루 최대 요청 수 (Requests Per Day)

- Usage Limits
  - 월간 사용량 제한
  - Set a Budget Alert : 알림 설정
  - Enable Budget Limit : 중단 설정

- Usage Tier
  - 조직에는 0 ~ 5 단계의 티어가 부여되며 최대 사용량 한도가 제한되어 있음
  - [Usage tiers Guide](https://platform.openai.com/docs/guides/rate-limits#usage-tiers)
  
- Usage 대시보드
  - 조직 수준 : `세팅 > Usage`
  - 프로젝트 수준 : `Dashboard > Usage`

### 3.1.6. 모델 종류 및 가격

- [모델 종류](https://platform.openai.com/docs/models#models-overview)
  - Context Window : 모델이 입력 받을 수 있는 최대 토큰수 (사용자 입력 토큰 + 모델이 생성한 토큰)
  - Max Output Tokens : 모델이 생성 가능한 최대 토큰
  - 모델명 : `모델-버전`
    > 버전은 일반적으로 날짜 형식이며, 생략된 경우 가장 최신 모델로 자동 호출 됨

- [모델 비용](https://openai.com/api/pricing/)
  - 모델 성능은 올라가고 점차 가격은 내려가는 추세
  - 이미지 (고해상도) : 512 x 512 px 단위로 토큰 가격이 매겨 짐
    - 저해상도의 경우 512 x 512 px 로 리사이
  - 오디오 : 분당 약 입력은 0.01 달러, 출력은 0.24 달러 (~~만세~~)
  - Batch API : 실시간 요청이 아닌 24시간 내 결과를 얻는 요청
  - Cached prompts : 최근에 요청이 존재할 경우 캐시된 응답 반환
  - Tool (Assistant API)
    - Code Interpreter : 최대 1시간 지속, 비용은 1회만 지불
    - File Search : 벡터 데이터베이스, 저장 용량만 지불

  


### 3.1.7. API 키 발급 (프로젝트)

- 세팅 > PROJECT > API keys > `+ Create new secret key`
- 휴대폰 인증 필요
- 입력 예시
  - Name (옵션) : Lecture Test
  - Project : 교육 실습 용 프로젝트 선택
  - Permissions : All
- `sk-proj-`로 시작하는 키 복사
- `.evn` 파일에 `OPENAI_API_KEY` 변수로 저장
    > `OPENAI_API_KEY=sk-proj-****`

## 3.2. GPT로 주행 예측하기

### 3.2.1. OpenAI 및 Vehicle API 클라이언트 로드

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from aicastle.deepracer.vehicle.api.client import VehicleClient
load_dotenv(".env")

openai = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY")
)

vehicle =VehicleClient(
    ip=os.getenv("VEHICLE_IP"),
    password=os.getenv("VEHICLE_PASSWORD"),
)

### 3.2.2. 변수 및 프롬프트 정의

In [122]:
from aicastle.deepracer.vehicle.api.utils import read_image, get_image_base64

image_width = 640
image_height = 480
image_resolution = "high"    # "auto", "high", "low"

messages_prompt = [
    {
        "role": "system",
        "content": [
            {
                "type": "text",
                "text": (
                    "You are an autonomous driving AI."
                    "Analyze the image captured by the front camera and determine the direction."
                    "Answer only with one of: straight, left, or right."
                )
            }
        ]
    },
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "차선을 이탈하거나 상자(장애물)과 충돌하지 않게 주행하세요."
            }
        ]
    },
    {
        "role": "user",
        "content": [
            {
                "type": "image_url", 
                "image_url": {
                    "url": "data:image/jpeg;base64,{}".format(
                        get_image_base64(
                            read_image(
                                'sample/images/1_right.png', 
                                width=image_width, 
                                height=image_height,
                                color="rgb"
                            ),
                            format="jpg"
                        )
                    ),
                    "detail":image_resolution
                }
            }
        ]
    },
    {
        "role": "assistant",
        "content": [
            {
                "type": "text",
                "text": "right"
            }
        ]
    },
    {
        "role": "user",
        "content": [
            {
                "type": "image_url", 
                "image_url": {
                    "url": "data:image/jpeg;base64,{}".format(
                        get_image_base64(
                            read_image(
                                'sample/images/2_straight.png',
                                width=image_width, 
                                height=image_height,
                                color="rgb"
                            ),
                            format="jpg"
                        )
                    ),
                    "detail":image_resolution
                }
            }
        ]
    },
    {
        "role": "assistant",
        "content": [
            {
                "type": "text",
                "text": "straight"
            }
        ]
    },
    {
        "role": "user",
        "content": [
            {
                "type": "image_url", 
                "image_url": {
                    "url": "data:image/jpeg;base64,{}".format(
                        get_image_base64(
                            read_image(
                                'sample/images/3_left.png',
                                width=image_width, 
                                height=image_height,
                                color="rgb"
                            ),
                            format="jpg"
                        )
                    ),
                    "detail":image_resolution
                }
            }
        ]
    },
    {
        "role": "assistant",
        "content": [
            {
                "type": "text",
                "text": "left"
            }
        ]
    },
]

### 3.2.3. 예측 및 자율주행

In [ ]:
from IPython.display import clear_output
import time

model = "gpt-4o"
repeat_count = 10
move_duration = 1
user_message = """

""".strip()

target_actions = {
    "straight":{"angle": 0, "speed": 2},
    "left":{"angle": 30, "speed": 2},
    "right":{"angle": -30, "speed": 2},
}

vehicle.set_speed_percent(60)
try :
    for repeat_idx in range(repeat_count):       
        # image = read_image("sample/images/test.png", width=image_width, height=image_height, color="rgb")
        image = vehicle.get_image(width=image_width, height=image_height, color="rbg")
        image_base64 = get_image_base64(image, format="jpg")
        image_url = f"data:image/jpeg;base64,{image_base64}"
        messages = messages_prompt + [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": user_message
                    },
                    {
                        "type": "image_url", 
                        "image_url": {
                            "url": image_url,
                            "detail":image_resolution
                        }
                    }
                ]
            }
        ]
        # messages = [{"role": "user", "content": "안녕?"}]

        # OpenAI에 요청하기
        s_time = time.time()
        response = openai.chat.completions.create(
            model = model,
            messages = messages,
            max_tokens=1,
            temperature=0,
            top_p=0,
            logprobs=True,
            top_logprobs=20,
        )
        f_time = time.time()

        content = response.choices[0].message.content
        action_probs = {top_logprob.token:math.exp(top_logprob.logprob) for top_logprob in response.choices[0].logprobs.content[0].top_logprobs}
        target_action_probs = {action: action_probs.get(action, 0) for action in target_actions}
        target_action_top = max(target_action_probs, key=target_action_probs.get)
        
        if target_action_probs[target_action_top] > 0:
            probs_sum = sum(target_action_probs.values())
            angle = sum([target_actions[action]["angle"] * prob for action, prob in target_action_probs.items()]) / probs_sum
            speed = sum([target_actions[action]["speed"] * prob for action, prob in target_action_probs.items()]) / probs_sum    
        else :
            angle = 0
            speed = 0     
        
        print(f"[repeat_idx] {repeat_idx+1}/{repeat_count}")
        print(f"[inference_time] {f_time - s_time:.4f} sec")
        print(f"[content] {content}")
        print(f"[target_action_top] {target_action_top}")
        print("[target_action_probs]")
        for action, prob in target_action_probs.items():
            print(f"- {action}: {prob:.4f}")
        print(f"[angle] {angle:.4f}")
        print(f"[speed] {speed:.4f}")
        clear_output(wait=True)

        if angle != 0 or speed != 0:
            vehicle.move(angle=angle, speed=speed)
            time.sleep(move_duration)
            vehicle.move(angle=0, speed=0)

    vehicle.stop()
except :
    vehicle.stop()
